# モジュールのインポート

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt

from tqdm import tqdm

import numpy as np
import pandas as pd

import datetime
import jpholiday

import torch
import torch.utils.data
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.ensemble import RandomForestRegressor as RFR
 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.kernel_ridge import KernelRidge

import pickle

# データの前処理

In [2]:
# 説明変数の数
nfeat = 8

# 生データの読み込み
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# numpy配列化
Xmat = np.array(train_data)[:,:nfeat]
ymat = np.array(train_data)[:,nfeat]

all_data = pd.concat([train_data,test_data])
meta = pd.get_dummies(all_data['weather']) 

Xmat = np.hstack([Xmat[:,:nfeat],meta.iloc[:len(train_data),:]]) #use four real-valued features and the dummy, and cast to float64 data type

ymat = np.float64(ymat)

C:\Users\Toru\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [3]:
# データ整形
X = Xmat.copy()
y = ymat.copy()
labels = np.empty((X.shape[0], 24))

for i in range(Xmat.shape[0]):
    # 8列目をdatetimeに変換
    dt = datetime.datetime.strptime(Xmat[i,7], '%Y-%m-%d %H:%M:%S')
    
    # 祝日は1,そうでなければ0
    X[i,0] = 0 if Xmat[i,0] == 'None' else 1
    
    X[i,5] = dt.month
    X[i,6] = dt.day
    X[i,7] = dt.hour
print(X.shape)

(37696, 19)


In [4]:
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=0)

# ニューラルネットワーク

## モデル構築

In [5]:
class Net(nn.Module):

    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(input_dim, 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50, 50)
        self.bn2 = nn.BatchNorm1d(50)
        self.fc3 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [6]:
# 訓練用
input = torch.from_numpy(np.array(X_train, dtype=np.float64)).float()
target = torch.from_numpy(np.array(y_train, dtype=np.float64)).float()
train = torch.utils.data.TensorDataset(input, target)
train_loader = torch.utils.data.DataLoader(train, batch_size=10**4, shuffle=True)

# 検証用
input = torch.from_numpy(np.array(X_valid, dtype=np.float64)).float()
target = torch.from_numpy(np.array(y_valid, dtype=np.float64)).float()
valid = torch.utils.data.TensorDataset(input, target)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=10**4, shuffle=True)

In [7]:
net = Net(X.shape[-1])
net.train()

Net(
  (fc1): Linear(in_features=19, out_features=50, bias=True)
  (bn1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (bn2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=50, out_features=1, bias=True)
)

In [8]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.03)

## 学習

In [9]:
"""
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

# ネットワークをGPUへ
net.to(device)

# ネットワークがある程度固定であれば、高速化させる
torch.backends.cudnn.benchmark = True

num_epoch = 200
for i in range(num_epoch):
    for input_train, target_train in tqdm(train_loader):
        # GPUが使えるならGPUにデータを送る
        input_train = input_train.to(device)
        target_train = target_train.to(device)
        optimizer.zero_grad() #勾配の初期化
        output = net(input_train) #アウトプットの生成
        loss = criterion(output, target_train) #loss関数
        loss.backward() 
        optimizer.step()
    if i%10==0:
        net.eval()
        loss_valid = []
        for input_valid, target_valid in tqdm(valid_loader):
            # GPUが使えるならGPUにデータを送る
            input_valid = input_valid.to(device)
            target_valid = target_valid.to(device)
            output = net(input_valid)
            loss_valid.append(criterion(output, target_valid).item())
        print("epoch: {}, train loss: {}, valid loss: {}".format(i, loss, sum(loss_valid)/len(loss_valid)))
        net.train()
"""

'\n# GPUが使えるかを確認\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nprint("使用デバイス：", device)\n\n# ネットワークをGPUへ\nnet.to(device)\n\n# ネットワークがある程度固定であれば、高速化させる\ntorch.backends.cudnn.benchmark = True\n\nnum_epoch = 200\nfor i in range(num_epoch):\n    for input_train, target_train in tqdm(train_loader):\n        # GPUが使えるならGPUにデータを送る\n        input_train = input_train.to(device)\n        target_train = target_train.to(device)\n        optimizer.zero_grad() #勾配の初期化\n        output = net(input_train) #アウトプットの生成\n        loss = criterion(output, target_train) #loss関数\n        loss.backward() \n        optimizer.step()\n    if i%10==0:\n        net.eval()\n        loss_valid = []\n        for input_valid, target_valid in tqdm(valid_loader):\n            # GPUが使えるならGPUにデータを送る\n            input_valid = input_valid.to(device)\n            target_valid = target_valid.to(device)\n            output = net(input_valid)\n            loss_valid.append(criterion(output, target_val

## モデルの保存 

In [10]:
# torch.save(net.state_dict(), "weight.h")

## 予測

In [11]:
# モデルのロード
net = Net(X.shape[-1])
net.eval()
param = torch.load('weight.h')
net.load_state_dict(param)

# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

# ネットワークをGPUへ
net.to(device)

# ネットワークがある程度固定であれば、高速化させる
torch.backends.cudnn.benchmark = True

loss_valid = []
for input_valid, target_valid in tqdm(valid_loader):
    # GPUが使えるならGPUにデータを送る
    input_valid = input_valid.to(device)
    target_valid = target_valid.to(device)
    output = net(input_valid)
    loss_valid.append(criterion(output, target_valid).item())
RMSE = (sum(loss_valid)/len(loss_valid))**0.5
print("RMSE:",RMSE)    

使用デバイス： cuda:0


  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Toru\Anaconda3\envs\tf_gpu\lib\site-packages\torch\nn\modules\loss.py:443: UserWarning: Using a target size (torch.Size([3770])) that is different to the input size (torch.Size([3770, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


RMSE: 1966.602082272873


# カーネル回帰

In [12]:
"""
clf = KernelRidge(alpha=1.0, kernel='rbf')
clf.fit(X_train, y_train)
"""

"\nclf = KernelRidge(alpha=1.0, kernel='rbf')\nclf.fit(X_train, y_train)\n"

In [13]:
clf = pickle.load(open("kr.h", 'rb'))

In [14]:
pred = clf.predict(X_valid)
RMSE = np.sqrt(((pred - y_valid)**2).mean())
print("RMSE:",RMSE)

RMSE: 1128.4736186061675


# SVR

In [15]:
"""
clf_svr = svm.SVR(kernel='rbf', C=1e3, gamma=0.1, epsilon=0.1)
clf_svr.fit(X_train, y_train)
"""

"\nclf_svr = svm.SVR(kernel='rbf', C=1e3, gamma=0.1, epsilon=0.1)\nclf_svr.fit(X_train, y_train)\n"

In [16]:
clf_svr = pickle.load(open("svr.h", 'rb'))

In [17]:
pred = clf_svr.predict(X_valid)
RMSE = np.sqrt(((pred - y_valid)**2).mean())
print("RMSE:",RMSE)

RMSE: 1121.0568363841385


# ランダムフォレスト

In [18]:
rg = RFR(n_jobs=-1, random_state=2525)
 
rg.fit(X_train, y_train)

C:\Users\Toru\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=2525, verbose=0,
                      warm_start=False)

In [19]:
pred = rg.predict(X_valid)
RMSE = np.sqrt(((pred - y_valid)**2).mean())
print(RMSE)

866.8352424245395


## グリッドサーチ

In [20]:
"""
search_params = {
    'n_estimators'      : [10, 50, 100],
    'max_features'      : [i for i in range(1,X_train.shape[1])],
    'random_state'      : [2525],
    'n_jobs'            : [1],
    'min_samples_split' : [10, 30, 50, 100],
    'max_depth'         : [10, 30, 50, 100]
}
 
gsr = GridSearchCV(
    RFR(),
    search_params,
    cv = 3,
    n_jobs = -1,
    verbose=True
)
 
gsr.fit(X_train, y_train)
"""

"\nsearch_params = {\n    'n_estimators'      : [10, 50, 100],\n    'max_features'      : [i for i in range(1,X_train.shape[1])],\n    'random_state'      : [2525],\n    'n_jobs'            : [1],\n    'min_samples_split' : [10, 30, 50, 100],\n    'max_depth'         : [10, 30, 50, 100]\n}\n \ngsr = GridSearchCV(\n    RFR(),\n    search_params,\n    cv = 3,\n    n_jobs = -1,\n    verbose=True\n)\n \ngsr.fit(X_train, y_train)\n"

In [21]:
gsr = pickle.load(open("rfr.h", 'rb'))

In [22]:
rg = gsr.best_estimator_
pred = rg.predict(X_valid)
RMSE = np.sqrt(((pred - y_valid)**2).mean())
print("RMSE:",RMSE)

RMSE: 822.2745335155589


# 考察

In [23]:
rg = RFR(n_jobs=-1, random_state=2525)
columns = ["holiday", "temperature", "rain_in_hour", "snow_in_hour", "clouds_cover", "month", "day", "hour", "weather"]
results = {}
for i in range(9):
    if i != 8:
        X_train_cut = np.delete(X_train, i, 1)
        X_valid_cut = np.delete(X_valid, i, 1)
    else:
        X_train_cut = np.delete(X_train, [i for i in range(8,19)], 1)
        X_valid_cut = np.delete(X_valid, [i for i in range(8,19)], 1)
    rg.fit(X_train_cut, y_train)
    pred = rg.predict(X_valid_cut)
    RMSE = np.sqrt(((pred - y_valid)**2).mean())
    results[columns[i]] = RMSE
print(sorted(results.items(), key=lambda x:x[1]))

C:\Users\Toru\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[('weather', 853.5717007074451), ('snow_in_hour', 860.7132309534511), ('holiday', 863.5621476985116), ('clouds_cover', 872.2250712411947), ('rain_in_hour', 875.807274081207), ('month', 899.2219499794479), ('day', 945.4113151440395), ('temperature', 1002.5083976793011), ('hour', 1645.67932426515)]
